In [1]:
from helper import *
from evaluate import *

import pandas as pd

ROWS = ["train", "dev", "test", "test_cross_validation"]
DOMAIN_ORDER = ["restaurant", "laptop", "hotel"]  # Definiere die gewünschte Reihenfolge
SUBTASK_ORDER = [2, 3]  # Erst Subtask 2, dann Subtask 3
SUBTASKS = [2, 3]

records_dataset_statistics = []

for subtask in SUBTASKS:
    for language, domain in VALID_LANGUAGES_DOMAINS:

        # train + dev
        for split in ["train", "dev"]:
            count = len(get_dataset(subtask, language, domain, split=split))
            records_dataset_statistics.append((split, domain, subtask, language, count))

        # test (optional)
        try:
            count_test = len(get_dataset(subtask, language, domain, split="test"))
            records_dataset_statistics.append(("test", domain, subtask, language, count_test))
        except:
            pass

        # test_cross_validation (als ganze Zahl)
        train_size = len(get_dataset(subtask, language, domain, split="train"))
        records_dataset_statistics.append((
            "test_cross_validation",
            domain,
            subtask,
            language,
            str(int(train_size * 0.2))
        ))

df_dataset_statistics = pd.DataFrame(
    records_dataset_statistics,
    columns=["split", "domain", "subtask", "language", "count"]
)

# Konvertiere domain zu Categorical mit gewünschter Reihenfolge
df_dataset_statistics["domain"] = pd.Categorical(
    df_dataset_statistics["domain"], 
    categories=DOMAIN_ORDER, 
    ordered=True
)

df_dataset_statistics = (
    df_dataset_statistics
        .pivot(index=["split", "domain"],
               columns=["subtask", "language"],
               values="count")
        .sort_index(level=["split", "domain"], key=lambda x: x.map({s: i for i, s in enumerate(ROWS)} if x.name == "split" else {d: i for i, d in enumerate(DOMAIN_ORDER)}))
)

# Spalten nach gewünschter Subtask-Reihenfolge sortieren (erst 2, dann 3)
df_dataset_statistics = df_dataset_statistics.reindex(
    columns=sorted(df_dataset_statistics.columns, key=lambda x: (SUBTASK_ORDER.index(x[0]), x[1]))
)

df_dataset_statistics = df_dataset_statistics.applymap(
    lambda x: f"{int(x):,}" if pd.notna(x) else "-"
)

# get values from left to right from top to bottom as 1D list
values_list_dataset_statistics = df_dataset_statistics.values.flatten().tolist()
df_dataset_statistics

/tmp/ipykernel_1496508/2400303632.py:63: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_dataset_statistics = df_dataset_statistics.applymap(


subtask                               2                                     \
language                            eng    jpn    rus    tat    ukr    zho   
split                 domain                                                 
train                 restaurant  2,284      -  1,240  1,240  1,240  6,050   
                      laptop      4,076      -      -      -      -  3,490   
                      hotel           -  1,600      -      -      -      -   
dev                   restaurant    200      -     48     48     48    300   
                      laptop        200      -      -      -      -    300   
                      hotel           -    200      -      -      -      -   
test_cross_validation restaurant    456      -    248    248    248  1,210   
                      laptop        815      -      -      -      -    698   
                      hotel           -    320      -      -      -      -   

subtask                               3                                     
language                            eng    jpn    rus    tat    ukr    zho  
split                 domain                                                
train                 restaurant  2,284      -  1,240  1,240  1,240  6,050  
                      laptop      4,076      -      -      -      -  3,490  
                      hotel           -  1,600      -      -      -      -  
dev                   restaurant    200      -     48     48     48    300  
                      laptop        200      -      -      -      -    300  
                      hotel           -    200      -      -      -      -  
test_cross_validation restaurant    456      -    248    248    248  1,210  
                      laptop        815      -      -      -      -    698  
                      hotel           -    320      -      -      -      -